In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV, cross_val_score, RandomizedSearchCV
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.decomposition import PCA
from torch.utils.data import TensorDataset, DataLoader
from scipy.stats import uniform, randint, loguniform


In [48]:
data_train = pd.read_csv("./train.csv")
data_test = pd.read_csv("./test.csv")
X_train = data_train.iloc[:, 6:]
X_test = data_test.iloc[:, 5:]

print(X_test)

"""
data = pd.concat([data_train, data_test], axis=0)
X = data.iloc[:, 6:]
classifiers = data.iloc[:, 1:4]
enc = OneHotEncoder()
data_oh = enc.fit_transform(classifiers).toarray()
data_oh = pd.DataFrame(data_oh)

# Nombre de lignes dans data_train
n_train = len(data_train)

# Séparer data en data_train et data_test
data_train_oh = data_oh.iloc[:n_train, :].reset_index(drop=True)
data_test_oh = data_oh.iloc[n_train:, :].reset_index(drop=True)
"""


        908.1     914.3     920.5     926.7     932.9     939.1     945.3  \
0    0.319735  0.295819  0.270899  0.243264  0.217097  0.192895  0.172856   
1    0.226916  0.207225  0.190766  0.168512  0.151020  0.133545  0.119390   
2    0.046564  0.045257  0.043468  0.040846  0.036622  0.031702  0.027777   
3    0.098065  0.093345  0.088876  0.082929  0.075919  0.070756  0.066671   
4    0.108564  0.100385  0.091454  0.080773  0.069861  0.060781  0.054413   
..        ...       ...       ...       ...       ...       ...       ...   
603  0.143281  0.123888  0.104409  0.082312  0.060735  0.040328  0.023318   
604  0.297497  0.278773  0.259028  0.237541  0.216082  0.194281  0.176280   
605  0.296863  0.283916  0.276850  0.268556  0.261995  0.251307  0.240764   
606  0.145413  0.142384  0.139033  0.136055  0.132526  0.129186  0.126506   
607  0.037615  0.037858  0.038315  0.037060  0.033851  0.030902  0.028158   

        951.5     957.7     963.8  ...    1620.5    1626.6    1632.8  \
0  

'\ndata = pd.concat([data_train, data_test], axis=0)\nX = data.iloc[:, 6:]\nclassifiers = data.iloc[:, 1:4]\nenc = OneHotEncoder()\ndata_oh = enc.fit_transform(classifiers).toarray()\ndata_oh = pd.DataFrame(data_oh)\n\n# Nombre de lignes dans data_train\nn_train = len(data_train)\n\n# Séparer data en data_train et data_test\ndata_train_oh = data_oh.iloc[:n_train, :].reset_index(drop=True)\ndata_test_oh = data_oh.iloc[n_train:, :].reset_index(drop=True)\n'

In [49]:
#spectrum_filtered = pd.DataFrame(savgol_filter(spectrum, 7, 3, deriv = 2, axis = 0))
#spectrum_filtered_st = zscore(spectrum_filtered, axis = 1)

y = data_train["PURITY"]/100

pca = PCA(n_components=16)
X_pca = pd.DataFrame(pca.fit_transform(X_train))
print(pca.explained_variance_ratio_) #pour trouver le nombre de composantes à garder
X_new = pd.concat([X_train, X_pca], axis=1)
X_new.columns = X_new.columns.astype(str)
standardizer = StandardScaler()
X_st = standardizer.fit_transform(X_new)

X_train, X_valid, y_train, y_valid = train_test_split(X_st, y, test_size=0.2, random_state=42)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32)
y_valid_tensor = torch.tensor(y_valid.values, dtype=torch.float32).reshape(-1, 1)

[8.63385983e-01 1.19426741e-01 1.42997513e-02 1.98910919e-03
 4.96149909e-04 1.60475196e-04 9.86566323e-05 4.91435637e-05
 3.35702776e-05 1.86516555e-05 1.37499171e-05 8.14845972e-06
 4.42312642e-06 3.64445868e-06 2.58705888e-06 2.06030889e-06]


In [50]:
X_pca = pd.DataFrame(pca.transform(X_test))
X_new = pd.concat([X_test, X_pca], axis=1)
X_new.columns = X_new.columns.astype(str)

X_st = standardizer.transform(X_new)
X_test_tensor = torch.tensor(X_st, dtype=torch.float32)

In [51]:
# Définir le modèle de réseau de neurones simple
class FeedForwardNN(nn.Module):

    def __init__(self, input_size, lin_layer_sizes,
                 outpout_size, lin_layer_dropouts, activation):
        
        super().__init__()
        
        if activation == 0:
            self.activation = nn.ReLU()
        elif activation == 1:
            self.activation = nn.SiLU()
        elif activation == 2:
            self.activation = nn.Tanh()
        elif activation == 3:
            self.activation = nn.LeakyReLU()

    
        # Linear Layers
        first_lin_layer = nn.Linear(input_size, lin_layer_sizes[0])
    
        self.lin_layers = nn.ModuleList([first_lin_layer] + [nn.Linear(lin_layer_sizes[i], lin_layer_sizes[i + 1]) for i in range(len(lin_layer_sizes) - 1)])
        
        #for lin_layer in self.lin_layers:
            #nn.init.kaiming_normal_(lin_layer.weight.data)
      
        # Output Layer
        self.outpout_layer = nn.Linear(lin_layer_sizes[-1], outpout_size)
        #nn.init.kaiming_normal_(self.outpout_layer.weight.data)
    
    
        # Dropout Layers
        self.dropout_layers = nn.ModuleList([nn.Dropout(rate) for rate,size in zip(lin_layer_dropouts,lin_layer_sizes)])

    def forward(self, x):
  
        for lin_layer, dropout_layer in zip(self.lin_layers, self.dropout_layers):

            x = lin_layer(x)
        
            x = self.activation(x)

            x = dropout_layer(x)
      
        x = self.outpout_layer(x)
        x = nn.Sigmoid()(x)
    
        return x

# Définir la classe NeuralNetRegressor
class NeuralNetRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, input_size, random_state, eta=0.001, max_epochs=100, batch=10, lin_layer_sizes = [50, 50],
                 outpout_size = 1, lin_layer_dropouts = [0.4, 0.4], activation = 0):
        self.input_size = input_size
        self.random_state = random_state
        self.eta = eta
        self.max_epochs = max_epochs
        self.batch = batch
        self.lin_layer_sizes = lin_layer_sizes
        self.outpout_size = outpout_size
        self.lin_layer_dropouts = lin_layer_dropouts
        self.activation = activation
        self.model = FeedForwardNN(input_size, lin_layer_sizes,
                 outpout_size, lin_layer_dropouts, activation)
        self.criterion = nn.MSELoss()
    
    def fit(self, X, y, do_print=False):
        optimizer = optim.Adam(self.model.parameters(), lr=self.eta)
        X_tensor = torch.tensor(X).clone().detach().float()
        y_tensor = torch.tensor(y).clone().detach().float()
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(dataset, batch_size=self.batch, shuffle=True)
        self.model.train()
        # Training loop
        for epoch in range(self.max_epochs):
            epoch_loss = 0.0
            for batch_X, batch_y in dataloader:
                optimizer.zero_grad()  # Reset gradients
                outputs = self.model(batch_X)  # Forward pass
                loss = self.criterion(outputs, batch_y)  # Compute loss
                loss.backward()  # Backward pass
                optimizer.step()  # Update parameters
                epoch_loss += loss.item()
            if do_print:
                print(f"Epoch {epoch+1}/{self.max_epochs}, Loss: {epoch_loss / len(dataloader)}")
        return self
    
    def predict(self, X):
        self.model.eval()
        X_tensor = torch.tensor(X, dtype=torch.float32)
        with torch.no_grad():
            outputs = self.model(X_tensor).flatten()
        return outputs.numpy()
    
    def parameters(self):
        return self.model.parameters()

In [52]:
# Initialiser le modèle
seed = 43
np.random.seed(seed)
torch.manual_seed(seed)

input_size = X_train_tensor.shape[1]
net = NeuralNetRegressor(input_size=input_size, random_state=43)

# Définir les paramètres pour GridSearch
params_dist = {
    'eta': loguniform(1e-4, 1e-1),
    'max_epochs': randint(50, 150),
    'batch': randint(32, 70),
    'lin_layer_sizes': [[randint.rvs(32, 128) for _ in range(randint.rvs(1, 4))]],  # Taille de 1 à 4 couches, entre 32 et 128 neurones par couche
    'lin_layer_dropouts': [[uniform.rvs(0, 0.5) for _ in range(randint.rvs(1, 4))]],  # Dropout entre 0 et 0.5 pour chaque couche
    'activation': randint(0, 4),
}

# Initialiser RandomizedSearchCV
random_search = RandomizedSearchCV(net, params_dist, refit=True, cv=5, random_state=43, scoring='neg_mean_squared_error', verbose=0)

# Entraîner le modèle avec GridSearch
random_grid_result = random_search.fit(X_train_tensor, y_train_tensor)
nouveau_model = random_grid_result.best_estimator_

print("Best MSE: %f using %s" % (random_grid_result.best_score_, random_grid_result.best_params_))

/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_1887/3852911308.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X).clone().detach().float()
/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_1887/3852911308.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_tensor = torch.tensor(y).clone().detach().float()
/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_1887/3852911308.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X, dtype=torch.float32)
/var/fol

Best MSE: -0.001380 using {'activation': 2, 'batch': 48, 'eta': 0.0003283958684902921, 'lin_layer_dropouts': [0.06669548209299414, 0.12029480998267439], 'lin_layer_sizes': [96], 'max_epochs': 109}


In [54]:
y_pred = nouveau_model.predict(X_valid_tensor)
y_pred_train = nouveau_model.predict(X_train_tensor)
predictions = nouveau_model.predict(X_test_tensor)*100

# Vérifiez les sorties du modèle
print("y_pred contains NaN:", np.isnan(y_pred).any())

# Calculer la MSE
mse = np.mean(((y_pred - y_valid)*100)**2)
print("MSE :", mse)

# Calculer le t_score
train_score = np.mean(np.abs((y_pred_train-y_train)*100<=5))
test_score = np.mean(np.abs((y_pred-y_valid)*100<=5))
print("t_score test :", train_score)
print("t_score train :", test_score)

ids = np.arange(1, len(predictions) + 1)

# Create a DataFrame for the output
output_df = pd.DataFrame({
    'ID': ids,
    'PURITY': predictions
})

# Save the DataFrame to a CSV file
output_df.to_csv('predictions.csv', index=False) 

"""
# Afficher la courbe d'apprentissage
import matplotlib.pyplot as plt
plt.plot(steps, learning_curve)
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Learning Curve')
plt.show()
"""

/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_1887/3852911308.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X, dtype=torch.float32)


y_pred contains NaN: False
MSE : 13.410847758546995
t_score test : 0.9538461538461539
t_score train : 0.9384615384615385


"\n# Afficher la courbe d'apprentissage\nimport matplotlib.pyplot as plt\nplt.plot(steps, learning_curve)\nplt.xlabel('Steps')\nplt.ylabel('Loss')\nplt.title('Learning Curve')\nplt.show()\n"

In [35]:
torch.save(nouveau_model.model, "model1.pth")

In [36]:
toto = torch.load("model1.pth", weights_only=False)
toto.eval()

FeedForwardNN(
  (activation): ReLU()
  (lin_layers): ModuleList(
    (0): Linear(in_features=72, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
  )
  (outpout_layer): Linear(in_features=50, out_features=1, bias=True)
  (dropout_layers): ModuleList(
    (0-1): 2 x Dropout(p=0.4, inplace=False)
  )
)